In [8]:
from collections import deque
from collections import defaultdict
import math
import numpy as np
import os

In [9]:
class Car:
    def __init__(self, streets):
        self.streets = streets
        self.num_streets = len(streets)
        self.next_street = 0
        self.time_to_intersection = 0
        self.finish_time = None
        # add to first intersection
        self.streets[self.next_street].queue.append(self)

    def __str__(self):
        return(f"""
        Car:
          next_intersection: {self.streets[self.next_street].end_intersection.id}
          time_to_intersection: {self.time_to_intersection}
          is_done: {self.is_done()}
        """)


class Street:
    def __init__(self, name, length, start_intersection, end_intersection):
        self.name = name
        self.length = length
        self.start_intersection = start_intersection
        self.end_intersection = end_intersection
        self.queue = deque([])

    def __str__(self):
        return (f"""
        Street:
          name: {self.name}
          length: {self.length}
          start_intersection: {self.start_intersection.id}
          end_intersection: {self.end_intersection.id}
          queue: {[str(car) for car in self.queue]}
        """)

class Intersection:
    def __init__(self, id):
        self.id = id
        self.incoming = {}
        self.outgoing = {}
        self.light = None

    def __str__(self):
        return(f"""
        Intersection:
          id: {self.id}
          incoming: {self.incoming}
          outgoing: {self.outgoing}
          light: {self.light}
        """)

    def add_incoming(self, street):
        self.incoming[street.name] = street
  
    def add_outgoing(self, street):
        self.outgoing[street.name] = street

In [44]:
class Solution:
    def __init__(self, duration, num_intersections, num_streets, num_cars, bonus):
        self.duration = duration
        self.num_intersections = num_intersections
        self.num_streets = num_streets
        self.num_cars = num_cars
        self.bonus = bonus
        self.intersections = {}
        self.t = 0

    def __str__(self):
        return(f"""
        Solution:
            duration: {self.duration}
            num_intersections: {self.num_intersections}
            num_streets: {self.num_streets}
            num_cars: {self.num_cars}
            bonus: {self.bonus}
            intersections: {str(self.intersections)}
        """)

    def round_robin(self, filename, streets, cars, intersections):
        with open("out/" + filename, 'w') as file:
            file.write(f"{len(intersections)}\n")
            for intersection in intersections.values():
                file.write(f"{intersection.id}\n")
                
                file.write(f"{(len(intersection.incoming.values()))}\n")
                for strt in intersection.incoming.values():
                    file.write(f'{strt.name} 1\n')

    def simple_load(self, filename, streets, cars, intersections):
        loads = self.calc_load(cars)
        with open("out/" + filename, 'w') as file:
            valid_ints = []
            for i in intersections.values():
                d = loads[i.id]
                if any(d.values()):
                    valid_ints.append(i)
            file.write(f"{len(valid_ints)}\n")
            
            for intersection in valid_ints:
                file.write(f"{intersection.id}\n")
                
                file.write(f"{len([ strt for strt in intersection.incoming.values() if (loads[intersection.id][strt.name] > 0)] )}\n")
                for strt in intersection.incoming.values():
                    if loads[intersection.id][strt.name] > 0:
                        file.write(f'{strt.name} {max(1, min(self.duration, int(math.sqrt(loads[intersection.id][strt.name]))))}\n')
        
    def ln_load(self, filename, streets, cars, intersections):
        loads = self.calc_load(cars)
        with open("out/" + filename, 'w') as file:
            valid_ints = []
            for i in intersections.values():
                d = loads[i.id]
                if any(d.values()):
                    valid_ints.append(i)
            file.write(f"{len(valid_ints)}\n")
            
            for intersection in valid_ints:
                file.write(f"{intersection.id}\n")
                
                file.write(f"{len([ strt for strt in intersection.incoming.values() if (loads[intersection.id][strt.name] > 0)] )}\n")
                for strt in intersection.incoming.values():
                    if loads[intersection.id][strt.name] > 0:
                        file.write(f'{strt.name} {max(1, min(self.duration, int(np.math.log(loads[intersection.id][strt.name], 2.86))))}\n')

    def calc_load(self, cars):
        loads = defaultdict(lambda: defaultdict(int))
        interest_rate = 1
        for c in cars:
            for i, s in enumerate(c.streets):
                loads[s.end_intersection.id][s.name] += 1

        return loads

In [45]:
def parse_input(filename):
    with open(filename, 'r') as file:
        solution = None
        cars = []
        streets = {}
        intersections = {}
        for i, line in enumerate(file.readlines()):
            if i == 0:
                [D, I, S, V, F] = line.strip().split(' ')
                solution = Solution(int(D),I,S,V,F)
            elif i <= int(S):
                # processing streets
                [start, end, name, length] = line.strip().split(' ')

                if start not in intersections:
                    intersections[start] = Intersection(start)
                if end not in intersections:
                    intersections[end] = Intersection(end)
                street = Street(name, length, intersections[start], intersections[end])
                streets[street.name] = street

                intersections[end].add_incoming(street)
                intersections[start].add_outgoing(street)

            else:
                # processing cars
                street_names = line.strip().split(' ')[1:]
                car_streets = [streets[street_name] for street_name in street_names]
                cars.append(Car(car_streets))

    return solution, cars, streets, intersections

def run(letter = None):
    filenames = ['data/a.txt',
                 'data/b.txt',
                 'data/c.txt',
                 'data/d.txt',
                 'data/e.txt',
                 'data/f.txt']
    letter_to_int = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5}
    int_to_letter = {0 : 'a', 1 : 'b', 2 : 'c', 3 : 'd', 4 : 'f', 5 : 'g'}

    if letter is not None:
        filenames = [filenames[letter_to_int[letter]]]
    # with os.scandir("testcases") as input_files:
    # for input_file in input_files:
            
    for idx, input_file in enumerate(filenames):
        out_filename = 'out_'+int_to_letter.get(idx)+'.txt'
        solution, cars, streets, intersections = parse_input(input_file)
        solution.ln_load(out_filename, streets, cars, intersections)

In [47]:
run()